In [28]:
import os,sys,glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('table.csv')

In [31]:
df.keys()

Index(['Id', 'Element', 'Ion', 'Binding family',
       'Amino acids or nucleotide residues names', 'All bound residues',
       'Is homomer', 'Number of bound amino acids or nucleotide residues',
       'Number of all bound residues', 'Number of coordinating chains',
       'Representative'],
      dtype='object')

In [71]:
df_homo = df[(df['Is homomer'] == 1) & ~(df['Amino acids or nucleotide residues names'].str.contains('DG')) & ~(df['Amino acids or nucleotide residues names'].str.contains('DT')) & ~(df['Amino acids or nucleotide residues names'].str.contains('DC')) & ~(df['Amino acids or nucleotide residues names'].str.contains('DA'))]#.tail(40)


In [73]:
df_homo['Amino acids or nucleotide residues names'].str.contains('DU').sum()
#'DG' in df['Amino acids or nucleotide residues names']

0

In [82]:
df_homo[df_homo['Number of coordinating chains']==4]

,Id,Element,Ion,Binding family,Amino acids or nucleotide residues names,All bound residues,Is homomer,Number of bound amino acids or nucleotide residues,Number of all bound residues,Number of coordinating chains,Representative
39,1NC7-MG-1,MG,Mg²⁺,T4,THR.,THR.,True,4,4,4,True
40,1NC7-MG-2,MG,Mg²⁺,L4,LEU.,LEU.,True,4,4,4,True
41,1NC7-MG-3,MG,Mg²⁺,T4,THR.,HOH.,True,4,2,4,True
1041,4GGH-MG-1,MG,Mg²⁺,N4,ASN.,ASN.HOH.,True,4,6,4,True
1042,4GGH-MG-2,MG,Mg²⁺,N4,ASN.,ASN.HOH.,True,4,6,4,False
...,...,...,...,...,...,...,...,...,...,...,...
21717,7WM2-K-2,K,K⁺,T4,THR.,THR.,True,4,4,4,True
21718,7WM2-K-3,K,K⁺,G4,GLY.,GLY.,True,4,4,4,False
21764,8A35-RB-1,RB,Rb⁺,T4,THR.,THR.,True,4,4,4,False
21765,8A35-RB-2,RB,Rb⁺,T4,THR.,THR.,True,4,4,4,False


In [111]:
df_homo['pdb_id'] = df_homo['Id'].str.split('-', expand=True)[0]

/tmp/ipykernel_3732043/988947585.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_homo['pdb_id'] = df_homo['Id'].str.split('-', expand=True)[0]


In [115]:
df_homo.head(50)

,Id,Element,Ion,Binding family,Amino acids or nucleotide residues names,All bound residues,Is homomer,Number of bound amino acids or nucleotide residues,Number of all bound residues,Number of coordinating chains,Representative,pdb_id
2,5LDB-MG-1,MG,Mg²⁺,T2,THR.,HOH.THR.,True,2,3,2,True,5LDB
3,2SHK-MG-1,MG,Mg²⁺,A1G2R2,ALA.ARG.GLY.,ALA.GLN.HOH.,True,5,5,2,False,2SHK
7,2R6T-MG-1,MG,Mg²⁺,D1I1,ASP.ILE.,ASP.ATP.ILE.,True,2,3,2,False,2R6T
8,2R6T-MG-2,MG,Mg²⁺,D1I1,ASP.ILE.,ASP.ATP.ILE.,True,2,3,2,False,2R6T
9,2R6T-MG-3,MG,Mg²⁺,D1I1,ASP.ILE.,ASP.ATP.ILE.,True,2,3,2,False,2R6T
10,2R6X-MG-1,MG,Mg²⁺,D1I1,ASP.ILE.,ASP.ATP.HOH.ILE.,True,2,4,2,False,2R6X
11,2R6X-MG-2,MG,Mg²⁺,D1I1,ASP.ILE.,ASP.ATP.HOH.ILE.,True,2,4,2,False,2R6X
12,2R6X-MG-3,MG,Mg²⁺,D1I1,ASP.ILE.,ASP.ATP.HOH.ILE.,True,2,4,2,False,2R6X
13,2R8D-MG-1,MG,Mg²⁺,D3,ASP.,ASP.HOH.,True,3,6,2,True,2R8D
14,2R8D-MG-2,MG,Mg²⁺,D3,ASP.,ASP.HOH.,True,3,6,2,False,2R8D


In [119]:
#df_homo['pdb_id'].str.contains('4HHB').sum()

0

In [124]:
df_homo.to_csv('df_homomer.csv',index=False)

In [140]:
from urllib.request import urlretrieve
import requests
from tqdm import tqdm
import validators

In [123]:
url = 'https://files.rcsb.org/download/2AE8.pdb'
file = 'datasets/rcsb_pdbs/2AE8.pdb'
urlretrieve(url, file)

('datasets/rcsb_pdbs/2AE8.pdb', <http.client.HTTPMessage at 0x7f4aa1a3b9a0>)

In [159]:
logs = {'pdb_id':[], 'status':[]}
for pdb_id in tqdm(df_homo['pdb_id'].unique()):
    logs['pdb_id'].append(pdb_id)
    url = f'https://files.rcsb.org/download/{pdb_id}.pdb'
    file = f'datasets/rcsb_pdbs/{pdb_id}.pdb'
    r = requests.get(url)
    if '404' in str(r):
        logs['status'].append(0)
        continue
    urlretrieve(url,file)
    logs['status'].append(1)
logs = pd.DataFrame(logs)

  4%|▍         | 138/3199 [01:05<23:28,  2.17it/s] 

In [ ]:
logs.to_csv('datasets.logs')

In [155]:
#pdb_id
validators.url("https://files.rcsb.org/download/4QYK.pdb")
url = "https://files.rcsb.org/download/4QYK.pdb"
url = "https://files.rcsb.org/download/2IEC.pdb"
requests.get(url)
r = requests.get(url)
